Question 2: Please add the following calculated columns to the dataframe (plus any others you found helpful). 

Dynamic Columns (These vary by loan and by calendar month):

- current_balance: The current balance outstanding for each loan and month.
- seasoning: The integer number of months since the loan was originated at each month.
- n_missed_payments: number of missed payments in a row.
- prepaid_in_month: a flag indicating that the borrower prepaid in a given month.
- default_in_month: a flag indicating that the borrower defaulted in a given month.
- recovery_in_month: a flag indicating that a recovery has been made post-default in a given month.
- is_recovery_payment: a flag indicating whether the associated payment has been made post-default.
- time_to_reversion: The integer number of months until the laon reverts. This is negative if the - loan is before reversion and 0 at the month of reversion.
- is_post_seller_purchsae_date: Is this time period after the seller purchased this loan.

In [27]:
import pandas as pd
from case_functions import consolidate_data

In [28]:
pd.options.display.float_format = "{:,.2f}".format

In [3]:
data = consolidate_data()

In [4]:
data['Payment_Made'] = data['Payment_Made'].fillna(0)

In [5]:
data.head()

,loan_id,level_1,Balance,Payment_Due,Payment_Made,origination_date,investor_1_acquisition_date,reversion_date,original_balance,product,pre_reversion_fixed_rate,post_reversion_boe_margin
0,1,2020-12-31,"150,876.00",NaN,0.00,2020-12-31,2020-12-31,2022-12-31,150876,1,0.02,0.04
1,1,2021-01-31,"150,876.00",249.96,249.96,2020-12-31,2020-12-31,2022-12-31,150876,1,0.02,0.04
2,1,2021-02-28,"150,876.00",249.96,249.96,2020-12-31,2020-12-31,2022-12-31,150876,1,0.02,0.04
3,1,2021-03-31,"150,876.00",249.96,249.96,2020-12-31,2020-12-31,2022-12-31,150876,1,0.02,0.04
4,1,2021-04-30,"150,876.00",249.96,249.96,2020-12-31,2020-12-31,2022-12-31,150876,1,0.02,0.04


In [6]:
data['payment_made_cumsum'] = data.groupby("loan_id")['Payment_Made'].cumsum()

In [7]:
data['current_balance'] = data['original_balance'] - data['payment_made_cumsum']

In [8]:
data['level_1'] - data['origination_date']  # This give us days

0         0 days
1        31 days
2        59 days
3        90 days
4       120 days
          ...   
42606    92 days
42607   122 days
42608   153 days
42609   183 days
42610   214 days
Length: 42611, dtype: timedelta64[ns]

In [9]:
data['seasoning'] = data['level_1'].dt.to_period('M').astype(int) - data['origination_date'].dt.to_period('M').astype(int)

Question/Assumption

- I am going to assume that a payment which not zero but less than the amount due is a missed payment.
- There is only 1 instance this happens.


In [10]:
data [ (data['Payment_Due'] > data['Payment_Made'] ) & data['Payment_Made']>0]

,loan_id,level_1,Balance,Payment_Due,Payment_Made,origination_date,investor_1_acquisition_date,reversion_date,original_balance,product,pre_reversion_fixed_rate,post_reversion_boe_margin,payment_made_cumsum,current_balance,seasoning
21680,905,2021-09-30,"148,652.00",476.93,280.89,2019-08-31,2020-12-31,2021-08-31,148652,1,0.02,0.04,"6,741.40","141,910.60",25


In [11]:
data['missed_payment'] = data['Payment_Due'] > data['Payment_Made']

In [12]:
data['not_missed'] = ~data['missed_payment']

In [13]:
#  = data.groupby("loan_id")['missed_payment'].cumsum()

In [14]:
# data.groupby('loan_id')['not_missed'].cumsum()

In [15]:
# (~data['missed_payment']).cumsum()

In [16]:
data['n_missed_payments'] = data['missed_payment'].groupby((~data['missed_payment']).cumsum()).cumsum()

In [17]:
data[data.n_missed_payments >=3]['loan_id'].unique()

array([   3,   38,   52,   89,   92,   93,  154,  178,  199,  208,  212,
        222,  227,  292,  299,  313,  338,  361,  372,  429,  445,  469,
        478,  512,  516,  577,  614,  641,  648,  666,  681,  685,  692,
        696,  702,  704,  706,  748,  835,  844,  846,  855,  864,  903,
        946,  974,  996, 1085, 1119, 1120, 1177, 1188, 1212, 1215, 1239,
       1331, 1372, 1386, 1397, 1430, 1433, 1560, 1567, 1606])

In [18]:
data[data.loan_id == 1606]

,loan_id,level_1,Balance,Payment_Due,Payment_Made,origination_date,investor_1_acquisition_date,reversion_date,original_balance,product,pre_reversion_fixed_rate,post_reversion_boe_margin,payment_made_cumsum,current_balance,seasoning,missed_payment,not_missed,n_missed_payments
41738,1606,2021-11-30,"215,105.00",NaN,0.00,2021-11-30,2021-11-30,2023-11-30,215105,2,0.03,0.02,0.00,"215,105.00",0,False,True,0
41739,1606,2021-12-31,"215,105.00",543.95,543.95,2021-11-30,2021-11-30,2023-11-30,215105,2,0.03,0.02,543.95,"214,561.05",1,False,True,0
41740,1606,2022-01-31,"215,105.00",543.95,543.95,2021-11-30,2021-11-30,2023-11-30,215105,2,0.03,0.02,"1,087.91","214,017.09",2,False,True,0
41741,1606,2022-02-28,"215,105.00",543.95,543.95,2021-11-30,2021-11-30,2023-11-30,215105,2,0.03,0.02,"1,631.86","213,473.14",3,False,True,0
41742,1606,2022-03-31,"215,105.00",543.95,0.00,2021-11-30,2021-11-30,2023-11-30,215105,2,0.03,0.02,"1,631.86","213,473.14",4,True,False,1
41743,1606,2022-04-30,"215,105.00",543.95,"1,087.91",2021-11-30,2021-11-30,2023-11-30,215105,2,0.03,0.02,"2,719.77","212,385.23",5,False,True,0
41744,1606,2022-05-31,"215,105.00",543.95,543.95,2021-11-30,2021-11-30,2023-11-30,215105,2,0.03,0.02,"3,263.73","211,841.27",6,False,True,0
41745,1606,2022-06-30,"215,105.00",543.95,543.95,2021-11-30,2021-11-30,2023-11-30,215105,2,0.03,0.02,"3,807.68","211,297.32",7,False,True,0
41746,1606,2022-07-31,"215,105.00",543.95,0.00,2021-11-30,2021-11-30,2023-11-30,215105,2,0.03,0.02,"3,807.68","211,297.32",8,True,False,1
41747,1606,2022-08-31,"215,105.00",543.95,0.00,2021-11-30,2021-11-30,2023-11-30,215105,2,0.03,0.02,"3,807.68","211,297.32",9,True,False,2


In [19]:
# data['missed_payment'].groupby((~data['missed_payment']).cumsum()).cumsum()

- prepaid_in_month: a flag indicating that the borrower prepaid in a given month.

Prepayment: The full and early prepayment of a mortgage.

In [20]:

data['prepaid_in_month'] = (data['Payment_Due'] < data['Payment_Made']) & (data['Balance'] == 0)


In [21]:
data[(data.prepaid_in_month == True)]

,loan_id,level_1,Balance,Payment_Due,Payment_Made,origination_date,investor_1_acquisition_date,reversion_date,original_balance,product,pre_reversion_fixed_rate,post_reversion_boe_margin,payment_made_cumsum,current_balance,seasoning,missed_payment,not_missed,n_missed_payments,prepaid_in_month
8,1,2021-08-31,0.00,249.96,"151,125.96",2020-12-31,2020-12-31,2022-12-31,150876,1,0.02,0.04,"152,875.68","-1,999.68",8,False,True,0,True
108,3,2022-07-31,0.00,372.13,"88,870.54",2018-02-28,2020-12-31,2020-02-28,89311,1,0.03,0.04,"100,838.98","-11,527.98",53,False,True,0,True
220,7,2022-11-30,0.00,165.84,"88,191.84",2021-06-30,2021-06-30,2023-06-30,88026,1,0.02,0.04,"90,845.20","-2,819.20",17,False,True,0,True
308,10,2021-10-31,0.00,205.00,"64,102.00",2017-06-30,2020-12-31,2019-06-30,63897,1,0.03,0.04,"73,214.78","-9,317.78",52,False,True,0,True
351,13,2021-05-31,0.00,109.09,"55,780.09",2021-04-30,2021-04-30,2023-04-30,55671,1,0.02,0.04,"55,780.09",-109.09,1,False,True,0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40343,1567,2021-12-31,0.00,0.00,"65,444.94",2020-03-31,2020-12-31,2022-03-31,80728,2,0.02,0.02,"67,573.37","13,154.63",21,False,True,0,True
41395,1594,2021-08-31,0.00,263.25,"150,689.25",2016-11-30,2020-12-31,2018-11-30,150426,2,0.02,0.02,"167,564.71","-17,138.71",57,False,True,0,True
41751,1606,2022-12-31,0.00,543.95,"129,665.97",2021-11-30,2021-11-30,2023-11-30,215105,2,0.03,0.02,"133,473.66","81,631.34",13,False,True,0,True
41968,1614,2022-09-30,0.00,307.20,"87,047.20",2016-01-31,2020-12-31,2018-01-31,86740,2,0.03,0.02,"101,316.97","-14,576.97",80,False,True,0,True


- default_in_month: a flag indicating that the borrower defaulted in a given month.
- Default: The borrower misses three payments in a row. Borrowers cannot be cured from defaults.

In [22]:
data['default_in_month'] = (data.n_missed_payments == 3)

- recovery_in_month: a flag indicating that a recovery has been made post-default in a given month.
- Recovery: Any payments made post being flagged as default.

In [24]:
data['default_in_month_cumsum'] = data.default_in_month.cumsum() 

In [26]:
data[data.loan_id == 1606]

,loan_id,level_1,Balance,Payment_Due,Payment_Made,origination_date,investor_1_acquisition_date,reversion_date,original_balance,product,...,post_reversion_boe_margin,payment_made_cumsum,current_balance,seasoning,missed_payment,not_missed,n_missed_payments,prepaid_in_month,default_in_month,default_in_month_cumsum
41738,1606,2021-11-30,"215,105.00",NaN,0.00,2021-11-30,2021-11-30,2023-11-30,215105,2,...,0.02,0.00,"215,105.00",0,False,True,0,False,False,63
41739,1606,2021-12-31,"215,105.00",543.95,543.95,2021-11-30,2021-11-30,2023-11-30,215105,2,...,0.02,543.95,"214,561.05",1,False,True,0,False,False,63
41740,1606,2022-01-31,"215,105.00",543.95,543.95,2021-11-30,2021-11-30,2023-11-30,215105,2,...,0.02,"1,087.91","214,017.09",2,False,True,0,False,False,63
41741,1606,2022-02-28,"215,105.00",543.95,543.95,2021-11-30,2021-11-30,2023-11-30,215105,2,...,0.02,"1,631.86","213,473.14",3,False,True,0,False,False,63
41742,1606,2022-03-31,"215,105.00",543.95,0.00,2021-11-30,2021-11-30,2023-11-30,215105,2,...,0.02,"1,631.86","213,473.14",4,True,False,1,False,False,63
41743,1606,2022-04-30,"215,105.00",543.95,"1,087.91",2021-11-30,2021-11-30,2023-11-30,215105,2,...,0.02,"2,719.77","212,385.23",5,False,True,0,False,False,63
41744,1606,2022-05-31,"215,105.00",543.95,543.95,2021-11-30,2021-11-30,2023-11-30,215105,2,...,0.02,"3,263.73","211,841.27",6,False,True,0,False,False,63
41745,1606,2022-06-30,"215,105.00",543.95,543.95,2021-11-30,2021-11-30,2023-11-30,215105,2,...,0.02,"3,807.68","211,297.32",7,False,True,0,False,False,63
41746,1606,2022-07-31,"215,105.00",543.95,0.00,2021-11-30,2021-11-30,2023-11-30,215105,2,...,0.02,"3,807.68","211,297.32",8,True,False,1,False,False,63
41747,1606,2022-08-31,"215,105.00",543.95,0.00,2021-11-30,2021-11-30,2023-11-30,215105,2,...,0.02,"3,807.68","211,297.32",9,True,False,2,False,False,63


- is_recovery_payment: a flag indicating whether the associated payment has been made post-default.

In [ ]:
# - is_recovery_payment: a flag indicating whether the associated payment has been made post-default.

In [ ]:
data[data.n_missed_payments > 2]['loan_id'].unique()

In [ ]:
data[data.loan_id == 1606]

In [ ]:
df = data[data.loan_id == 1606]
df.missed_payment

- time_to_reversion: The integer number of months until the laon reverts. This is negative if the - loan is before reversion and 0 at the month of reversion.